In [1]:
import os
import re
import time
import random
import threading
import glob
import sys
import requests

In [53]:
class Move_ts_crawler(object):
    my_headers = [ "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 ",
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3314.0 Safari/537.36 SE 2.X MetaSr 1.0']
    
    def __init__(self, url, ts_url, file_name, video_name):
        self.url = url
        self.ts_url = ts_url
        self.file_name = file_name
        self.video_name = video_name
        self.header = {'User-Agent':Move_ts_crawler.my_headers[random.randrange(0, len(Move_ts_crawler.my_headers))]}
        self.lock = threading.RLock()
        self.__check_file()
        
    def __check_file(self):
        '''
        检查文件目录是否创建
        '''
        if not os.path.exists(self.file_name):
            os.makedirs(self.file_name)
        if os.path.exists(self.file_name + 'a.txt'):
            os.remove(self.file_name + 'a.txt')
            
    def down(self, url, name):
        '''
        下载ts文件
        '''
        self.header = {'User-Agent':Move_ts_crawler.my_headers[random.randrange(0, len(Move_ts_crawler.my_headers))]}
        response = requests.get(url, self.header)
        if response.status_code != 200:
            print("!!!!!!错误" + str(response.status_code) + url)
            quit()
        with open(self.file_name+name, 'wb') as f:
            f.write(response.content)
        response.close()
        
            
    def __merge_files(self):
        '''
        合并所有ts文件为mp4文件
        '''
        print('开始合并文件...')
        cmd = r'ffmpeg -y -f concat -safe 0 -i ' + self.file_name + 'a.txt -c copy ' + self.file_name + 'video.mp4'
        if os.system(cmd):
            print('合并失败')
            print(cmd)
        else:
            print('合并成功,删除其他文件...')
            self.__del_files()
        
    def __del_files(self):
        '''
        将所有ts文件删除,并将保存文件路径的txt文件删除,同时修改mp4文件名
        '''
        for i in glob.glob(os.path.join(self.file_name, '*.ts')):
            os.remove(i)
        os.remove(self.file_name + 'a.txt')
        src = self.file_name+'video.mp4'
        dst = self.file_name+self.video_name+'.mp4'
        if os.path.exists(dst):
            os.remove(dst)
        os.rename(src, dst)
        print('删除完成,视频制作完成')
        print('视频地址: ' + dst)
        
    
    def start(self):
        print('开始下载>>>>>')
        response = requests.get(self.url, self.header)
        text = response.content.decode('utf8')
        patten = re.compile(r'\n(.*?)\d+\.ts')
        prefix = patten.search(text)
        if prefix:
            prefix = prefix.group(1)
        self.prefix = prefix
        patten = re.compile(r'\d+\.ts')
        name_list = patten.findall(text)
        count = 1
        old_progress = -10
        for name in name_list:
            ts_url = self.ts_url + '/' + self.prefix + name
            with open(self.file_name+'a.txt', 'a+') as f:
                f.write('file ' + name + '\n')
            f.close()
            t = threading.Thread(target=self.down, args=(ts_url, name))
            t.start()
            while threading.active_count() >30:
                pass
            progress = int(count/len(name_list)*100)
            if progress > old_progress + 10:
                print('文件已下载{}%'.format(progress))
                old_progress = progress
            count += 1
        t.join()
        print('文件下载完成,开始处理....')
        self.__merge_files()

Exception in thread Thread-8201:
Traceback (most recent call last):
  File "E:\Anaconda\Anaconda\lib\site-packages\urllib3\contrib\pyopenssl.py", line 297, in recv_into
    return self.connection.recv_into(*args, **kwargs)
  File "E:\Anaconda\Anaconda\lib\site-packages\OpenSSL\SSL.py", line 1822, in recv_into
    self._raise_ssl_error(self._ssl, result)
  File "E:\Anaconda\Anaconda\lib\site-packages\OpenSSL\SSL.py", line 1639, in _raise_ssl_error
    raise SysCallError(errno, errorcode.get(errno))
OpenSSL.SSL.SysCallError: (10054, 'WSAECONNRESET')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\Anaconda\Anaconda\lib\site-packages\urllib3\response.py", line 360, in _error_catcher
    yield
  File "E:\Anaconda\Anaconda\lib\site-packages\urllib3\response.py", line 442, in read
    data = self._fp.read(amt)
  File "E:\Anaconda\Anaconda\lib\http\client.py", line 457, in read
    n = self.readinto(b)
  File "E:\Anaconda\Anac

In [52]:
# 进入网页,F12查找m3u8文件,部分网页可能有多个,找到那个内部含有ts文件名的,将地址复制
url = "https://meng.wuyou-zuida.com/20200326/28491_8fa733e9/1000k/hls/index.m3u8"
video_name = '喋血战士'
ts_url = '/'.join(url.split('/')[0:-1])
file_name = r'G:/Video/BloodShot/'
ts = Move_ts_crawler(url, ts_url, file_name, video_name)
ts.start()

开始下载>>>>>
文件已下载1%
文件已下载12%
文件已下载23%
文件已下载34%
文件已下载45%
文件已下载56%
文件已下载67%
文件已下载78%
文件已下载89%
文件已下载100%
文件下载完成,开始处理....
开始合并文件...
合并成功,删除其他文件...


FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'G:/Video/BloodShot/video.mp4' -> 'G:/Video/BloodShot/喋血战士.mp4'